In [2]:
# Import libraries and modules
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from tensorflow.keras.preprocessing.image import ImageDataGenerator
np.random.seed(42)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras import utils
from tensorflow.keras.datasets import mnist

In [3]:
colorectalx = pd.read_csv('../data/hmnist_64_64_L.csv')
colorectalx.drop(columns = 'label', inplace=True)
colorectaly = pd.read_csv('../data/hmnist_64_64_L.csv')

In [4]:

X = colorectalx
y = colorectaly['label']

In [5]:
y.value_counts(normalize = True)

7    0.125
3    0.125
6    0.125
2    0.125
5    0.125
1    0.125
8    0.125
4    0.125
Name: label, dtype: float64

In [6]:
X = X / 255.0 

In [8]:
# Reshaping
X = X.values.reshape(-1,64,64,1)       # shaping for the Keras
y = y.values

In [9]:
y = utils.to_categorical(y)

In [45]:
# split the data into the training and testing sets

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state=42)


In [46]:
print("x_train.shape: ",X_train.shape)
print("x_val.shape: ",X_val.shape)
print("y_train.shape: ",y_train.shape)
print("y_val.shape: ",y_val.shape)

x_train.shape:  (4000, 64, 64, 1)
x_val.shape:  (1000, 64, 64, 1)
y_train.shape:  (4000, 9)
y_val.shape:  (1000, 9)


In [47]:
# Check shape of an image.
X_train.shape

(4000, 64, 64, 1)

In [48]:
# Check shape of an image.
X_train[0].shape

(64, 64, 1)

In [49]:
# Instantiate a CNN.
cnn_model_2 = Sequential()

In [51]:
# Add a convolutional layer.
cnn_model_2.add(Conv2D(filters=128,             # number of filters
                       kernel_size=(5,5),      # height/width of filter
                       activation='relu',
                       padding = 'same',# activation function 
                       input_shape=(64,64,1))) # shape of input (image)

In [52]:
# Add a pooling layer.
cnn_model_2.add(MaxPooling2D(pool_size=(2,2))) # dimensions of region of pooling

In [54]:
cnn_model_2.add(Dropout(0.25))

In [55]:
# Add another convolutional layer.
cnn_model_2.add(Conv2D(64,
                       kernel_size=(3,3),
                       activation='relu'))

In [56]:
# Add another pooling layer.
cnn_model_2.add(MaxPooling2D(pool_size=(2,2)))

In [57]:
cnn_model_2.add(Dropout(0.25))

In [58]:
# Add another convolutional layer.
cnn_model_2.add(Conv2D(64,
                       kernel_size=(3,3),
                       activation='relu'))

# Add another pooling layer.
cnn_model_2.add(MaxPooling2D(pool_size=(2,2)))

cnn_model_2.add(Dropout(0.25))

In [59]:
cnn_model_2.add(Flatten())

In [60]:
cnn_model_2.add(Dense(256, activation='relu'))
cnn_model_2.add(Dense(64, activation='relu'))
cnn_model_2.add(Dense(32, activation='relu'))

In [61]:
cnn_model_2.add(Dense(9, activation='softmax'))

In [63]:
cnn_model_2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 60, 60, 128)       3328      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 60, 60, 128)       409728    
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 28, 28, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 64)        0

In [66]:
# Compile model
cnn_model_2.compile(loss='categorical_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy'])



In [71]:
datagen = ImageDataGenerator(
        rotation_range=0.5, 
        zoom_range = 0.5, 
        width_shift_range=0.5,  
        height_shift_range=0.5, 
        horizontal_flip=True, 
        vertical_flip=True)

datagen.fit(x_train)

In [74]:
cnn_model_2.fit_generator(datagen.flow(x_train,y_train, batch_size=200),
                              epochs = 20, validation_data = (x_val,y_val))

Epoch 1/20
20/20 [==============================] - 1087s 55s/step - loss: 1.2724 - accuracy: 0.4857 - val_loss: 1.1236 - val_accuracy: 0.5550
Epoch 2/20
20/20 [==============================] - 1114s 56s/step - loss: 0.9851 - accuracy: 0.5990 - val_loss: 0.9945 - val_accuracy: 0.6190
Epoch 3/20
20/20 [==============================] - 1129s 57s/step - loss: 0.9909 - accuracy: 0.5987 - val_loss: 1.1946 - val_accuracy: 0.4730
Epoch 4/20
20/20 [==============================] - 564s 28s/step - loss: 0.9811 - accuracy: 0.5993 - val_loss: 0.9805 - val_accuracy: 0.6330
Epoch 5/20
20/20 [==============================] - 562s 28s/step - loss: 0.9262 - accuracy: 0.6235 - val_loss: 0.9537 - val_accuracy: 0.6390
Epoch 6/20
20/20 [==============================] - 558s 28s/step - loss: 0.8996 - accuracy: 0.6410 - val_loss: 1.1650 - val_accuracy: 0.5760
Epoch 7/20
20/20 [==============================] - 576s 29s/step - loss: 0.8934 - accuracy: 0.6492 - val_loss: 0.9790 - val_accuracy: 0.6330
Epo

In [65]:
# Fit model on training data
history = cnn_model_2.fit(X_train,
                          y_train,
                          batch_size=200,
                          validation_data=(X_val, y_val),
                          epochs=20,
                          verbose=1)

Epoch 1/20
20/20 [==============================] - 791s 39s/step - loss: 2.1039 - accuracy: 0.1486 - val_loss: 1.7915 - val_accuracy: 0.2320
Epoch 2/20
20/20 [==============================] - 628s 31s/step - loss: 1.7124 - accuracy: 0.2656 - val_loss: 1.3528 - val_accuracy: 0.5040
Epoch 3/20
20/20 [==============================] - 502s 25s/step - loss: 1.3154 - accuracy: 0.4330 - val_loss: 1.3294 - val_accuracy: 0.3930
Epoch 4/20
20/20 [==============================] - 498s 25s/step - loss: 1.1949 - accuracy: 0.4844 - val_loss: 1.2083 - val_accuracy: 0.4760
Epoch 5/20
20/20 [==============================] - 507s 25s/step - loss: 1.0619 - accuracy: 0.5479 - val_loss: 1.1516 - val_accuracy: 0.5000
Epoch 6/20
20/20 [==============================] - 481s 24s/step - loss: 1.0555 - accuracy: 0.5513 - val_loss: 1.1254 - val_accuracy: 0.4540
Epoch 7/20
20/20 [==============================] - 484s 24s/step - loss: 1.1470 - accuracy: 0.5331 - val_loss: 1.0520 - val_accuracy: 0.5790
Epoch 